# Հայկական փոփ երգերի բառերի քերում

In [1]:
import scrapy as քերիչ
from scrapy.crawler import CrawlerProcess as ՍարդԳործընթաց

In [3]:
# Այստեղ կպահվեն քերված տվյալները
արտիստներ = []
բառեր = []
անուններ = []

class ՄովսիսյանիՓոփՍարդ(քերիչ.Spider):
    name = "ՄովսիսյանիՓոփՍարդ"
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.domain = "https://lyricstranslate.com" 
        self.scope = self.domain + "/en/songs/321/none/none/0"

    def start_requests(self):
        urls = [self.scope] + \
            [self.scope + f"?page={i}" for i in range(1, 11)]
        
        for url in urls:
            yield քերիչ.Request(url=url, callback=self.parse_page)

    def parse_page(self, response):
        links_current = response.xpath('//td[@class="ltsearch-songtitle"]/a/@href').extract()
        i = 1

        for link in links_current:
            yield response.follow(url=link, callback=self.parse_song)



    def parse_song(self, response):
        artist = response.css('li.song-node-info-artist > a::text').extract_first()
        name = response.css('div.song-node-text > h2.title-h2::text').extract_first().strip()
        lyric = " ".join([i.strip() for i in response.css('div.ltf > div.par ::text').extract()])

        արտիստներ.append(artist)
        անուններ.append(name)
        բառեր.append(lyric)

In [4]:
գործընթաց = ՍարդԳործընթաց()
գործընթաց.crawl(ՄովսիսյանիՓոփՍարդ)
գործընթաց.start()

2021-09-13 19:34:36 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-09-13 19:34:36 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.9.1 (tags/v3.9.1:1e5d33e, Dec  7 2020, 17:08:21) [MSC v.1927 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Windows-8.1-6.3.9600-SP0
2021-09-13 19:34:36 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-09-13 19:34:36 [scrapy.crawler] INFO: Overridden settings:
{}
2021-09-13 19:34:36 [scrapy.extensions.telnet] INFO: Telnet Password: 42fbc7ac50f4192b
2021-09-13 19:34:36 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-09-13 19:34:37 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddle

In [5]:
len(արտիստներ), len(անուններ), len(բառեր)

(1050, 1050, 1050)

In [6]:
import pandas as pd

կրիչ = pd.DataFrame(dict(արտիստ = արտիստներ, անուն = անուններ, բառեր = բառեր))

In [7]:
կրիչ

,արտիստ,անուն,բառեր
0,Saro,Mi G'na,Yepr arachin ankam tesa kez kyanke im mi hekya...
1,Silva Hakobyan,Սասնա Պար,"Դը քելե՛, հա՛յ, դը քելե՛, Դը քելե՛, քելե՛, քել..."
2,Sona Rubenyan,Աշնան Սերենադ,"Օրը մթնեց դանդաղ Երբ ինձնից դու հեռացար անհետ,..."
3,Arevik,Erazanq,Մեծ երկնքում փայլող աստղեր Ձեզ խնդրում եմ ինձ ...
4,Sevak Amroyan,Axpers u es,"Անօրեն թուրքն է տիրել մայր հողին, Ի՞նչ սրտով պ..."
...,...,...,...
1045,Zara (Russia),Դլե Յաման,"Դլե յաման Գյամին էկավ կրակի պես, Վա՜յ, դլե յամ..."
1046,Serj Tankian,Բարի Արագիլ,"Ես ոչ անտուն եմ, ոչ էլ տարագիր, Ունեմ հանգրվան..."
1047,Razmik Amyan,Sirem qez lianam,"Գտել եմ քեզ հիմա, ու էլ երբեք չեմ թողնի Գտել ե..."
1048,Super Sako,Մի Գնա,Մենակ եմ աշխարում Առանց քեզ (Պապայի արև) Տենչո...


In [9]:
# Հեռացնում ենք ոչ-հայատառ տեքստերը
կրիչ = կրիչ[կրիչ.բառեր.str.contains("ա")]
կրիչ

,արտիստ,անուն,բառեր
1,Silva Hakobyan,Սասնա Պար,"Դը քելե՛, հա՛յ, դը քելե՛, Դը քելե՛, քելե՛, քել..."
2,Sona Rubenyan,Աշնան Սերենադ,"Օրը մթնեց դանդաղ Երբ ինձնից դու հեռացար անհետ,..."
3,Arevik,Erazanq,Մեծ երկնքում փայլող աստղեր Ձեզ խնդրում եմ ինձ ...
4,Sevak Amroyan,Axpers u es,"Անօրեն թուրքն է տիրել մայր հողին, Ի՞նչ սրտով պ..."
5,Inga & Anush,Իմ Անունը Հայաստան Է,"Իմ անունը Հայաստան է, Ես ազատ եմ ու անկախ Իմ ա..."
...,...,...,...
1045,Zara (Russia),Դլե Յաման,"Դլե յաման Գյամին էկավ կրակի պես, Վա՜յ, դլե յամ..."
1046,Serj Tankian,Բարի Արագիլ,"Ես ոչ անտուն եմ, ոչ էլ տարագիր, Ունեմ հանգրվան..."
1047,Razmik Amyan,Sirem qez lianam,"Գտել եմ քեզ հիմա, ու էլ երբեք չեմ թողնի Գտել ե..."
1048,Super Sako,Մի Գնա,Մենակ եմ աշխարում Առանց քեզ (Պապայի արև) Տենչո...


In [11]:
կրիչ.արտիստ.value_counts()

Ruben Hakhverdyan     93
Armenian Folk         76
Arthur Meschian       22
Lilit Hovhannisyan    21
Hovhannes Tumanyan    21
                      ..
Davit Amalyan          1
Gilbert Hovsepian      1
Lydia Dervartian       1
Chinar Isoyan          1
Heda Hamzatova         1
Name: արտիստ, Length: 275, dtype: int64

In [12]:
կրիչ.բառեր.str.len().mean()

827.9264392324094